<a href="https://www.kaggle.com/code/samerattrah/turtlevision-challenge?scriptVersionId=268094607" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Jellyfish and Plastic Pollution Classification

This notebook demonstrates a transfer learning approach using InceptionV3 to classify images of different types of jellyfish and plastic pollution. The process involves loading the dataset, fine-tuning a pre-trained InceptionV3 model, and generating predictions on a test set.

This cell imports the necessary libraries for the project, including TensorFlow, Keras Tuner, NumPy, OS, Pathlib, Matplotlib, IPython, Sys, OpenCV, and CSV.

In [ ]:
# This cell imports the necessary libraries for the project.

import tensorflow as tf
import keras_tuner
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import IPython
import sys
import cv2
import csv

This cell loads the training, validation, and test datasets using `tf.keras.utils.image_dataset_from_directory`. It specifies the data paths, image size, color mode, batch size, and splits the training data into training and validation sets.

In [ ]:
# This cell loads the training, validation, and test datasets.

# Dataset loading
data_path = pathlib.Path('/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/train/')
data_path_test = pathlib.Path('/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/test/')
AUTOTUNE = tf.data.AUTOTUNE

# Training and validation datasets
dataset_path, dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels='inferred',
    validation_split=0.2,
    subset='both',
    seed=1,
    batch_size=16,
    image_size=(244, 244),
    color_mode="rgb",
    shuffle=True
)

# Test dataset
dataset_path_test = tf.keras.utils.image_dataset_from_directory(
    data_path_test,
    labels='inferred',
    seed=2,
    batch_size=1,
    image_size=(244, 244),
    color_mode="rgb",
    shuffle=True
)

This cell downloads the pre-trained InceptionV3 model weights from ImageNet. The `include_top=False` argument removes the classification layer, allowing us to fine-tune the model for our specific task.

In [ ]:
# This cell downloads the pre-trained InceptionV3 model weights from ImageNet.

# downloading the weights of the base model
base_model = tf.keras.applications.InceptionV3(
    input_shape=(244, 244, 3),
    include_top=False,  # Remove the classification layer
    weights="imagenet"
)

This cell saves the downloaded base model to a file named 'SavedBaseModel.h5'. This allows us to load the model later without re-downloading the weights.

In [ ]:
# This cell saves the downloaded base model to a file.

# saving the downloaded base_model
saving_path = pathlib.Path('SavedBaseModel.h5')
base_model.save(saving_path)

This cell loads the saved base model, sets it as non-trainable, and defines data preprocessing and augmentation layers. It then builds the classification model by adding a global average pooling layer and a dense output layer with softmax activation. The model is compiled with an Adam optimizer and sparse categorical crossentropy loss. It is then trained for 15 epochs with the base model frozen. Afterwards, the base model is made trainable, and the batch normalization layers are frozen. The model is recompiled with a lower learning rate and trained for another 35 epochs.

In [ ]:
# This cell loads and trains the model.

#Model
model_path = pathlib.Path('SavedBaseModel.h5')

# Loading base_model
base_model = tf.keras.models.load_model(model_path)

# Setting the base model as non-trainable initially
base_model.trainable = False

# Data preprocessing and augmentation layers
rescaling = tf.keras.Sequential([
    # Rescaling to (1, -1) range required for inceptionV3 model
    tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
])
augmentation = tf.keras.Sequential([
    # Applying augmentations on the images
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1)
])

# Build the classification model
inputs = tf.keras.Input(shape=(244, 244, 3))
# x = rescaling(inputs) # Optional: apply rescaling
# x = augmentation(x) # Optional: apply augmentation
x = tf.keras.applications.inception_v3.preprocess_input(inputs) # Preprocess input for InceptionV3
x = base_model(x, training=False)  # Pass through the base model
x = tf.keras.layers.GlobalAveragePooling2D()(x) # Add a global average pooling layer
outputs = tf.keras.layers.Dense(6, activation='softmax')(x)  # Add a dense output layer with softmax activation
model = tf.keras.Model(inputs, outputs)

# Compile the model with a low learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00029979988271190114),
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly=True  # Set to True for easier debugging
)

  # return model

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_loss",
#     max_trials=6,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=20, validation_data = dataset_path_val)

# tuner.results_summary()


model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=15,
    validation_data=dataset_path_val
)

model.summary()

# Switch the base_model to trainable for fine-tuning
base_model.trainable = True

# Freeze all batchnormalization layers of the base_model to not lose weights
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly=True  # Set to True for easier debugging
)

# Fit the model for another 35 epochs with the base model unfrozen
model.fit(
    dataset_path,
    epochs=50,  # Total epochs = 15 + 35
    initial_epoch=15, # Start from epoch 15
    validation_data=dataset_path_val
)

# model.evaluate(dataset_path_test, batch_size=5, verbose=1) # Optional: evaluate on test set

model.summary()

This cell saves the trained model to a specified directory in the TensorFlow SavedModel format. This allows the model to be loaded and used later for making predictions.

In [ ]:
# This cell saves the trained model.

# save model
saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/saved_model/')

tf.keras.models.save_model(model,
                           saving_path,
                           overwrite=True,
                           save_format='tf'  # Save in TensorFlow SavedModel format
                           )

This cell loads the saved model from the specified directory and compiles it. The `compile=True` argument ensures that the model is ready for making predictions.

In [ ]:
# This cell loads the saved model for prediction.

# load and compile prediction model
loading_path = pathlib.Path('/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/saved_model/')
loaded_model = tf.keras.models.load_model(loading_path, compile=True)

# Compile the loaded model
loaded_model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

This cell defines a function `csv_writer` that takes a filename, fields (header row), and data as input and writes the data to a CSV file. It appends to the file if it already exists.

In [ ]:
# This cell defines a function to write data to a CSV file.

def csv_writer(filename, fields, data):
    """
    Writes data to a CSV file.

    Args:
        filename (str): The name of the CSV file.
        fields (list): A list of strings representing the header row.
        data (list of tuples): A list of tuples, where each tuple represents a row of data.

    Returns:
        bool: True if the data was successfully written to the file.
    """
    csvfile = filename
    with open(csvfile, mode="a", newline='') as first:  # Use mode="a" to append, newline='' to prevent extra blank rows
        csvwriter = csv.writer(first)
        csvwriter.writerow(fields)  # Write the header row
        csvwriter.writerows(data)  # Write the data rows

    return True

This cell iterates through the test dataset, loads each image, preprocesses it, and makes a prediction using the loaded model. It then maps the predicted class index to the corresponding class name and appends the image filename and predicted class to a list. Finally, it uses the `csv_writer` function to write the predictions to a CSV file named "TurtleVisionChallenge_Predictions_fineTuned.csv".

In [ ]:
# This cell generates predictions on the test set and writes them to a CSV file.

# file creation for the predicted labels list
submit = []
pred_data_path = pathlib.Path("/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/test/")

# Iterate through the test dataset
for dir1 in os.listdir(pred_data_path):
    for file in os.listdir(os.path.join(pred_data_path, dir1)):
        image_path = os.path.join(pred_data_path, dir1, file)

        # Load and preprocess the image
        image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        image = np.array(image)
        image = image.astype("float32")
        pred_image = np.reshape(image, (1, 244, 244, 3))

        # Make a prediction
        predictions = loaded_model.predict(pred_image)
        pred = np.argmax(predictions, axis=1)
        print("this is pred: ", pred)

        # Map the predicted class index to the class name
        if pred == 0:
            PredictedClass = "'barrel_jellyfish'"
            print("barrel_jellyfish")
        elif pred == 1:
            PredictedClass = "'compass_jellyfish'"
            print("compass_jellyfish")
        elif pred == 2:
            PredictedClass = "'lions_mane_jellyfish'"
            print("lions_mane_jellyfish")
        elif pred == 3:
            PredictedClass = "'mauve_stinger_jellyfish'"
            print("mauve_stinger_jellyfish")
        elif pred == 4:
            PredictedClass = "'moon_jellyfish'"
            print("moon_jellyfish")
        elif pred == 5:
            PredictedClass = "'plastic_pollution'"
            print("plastic_pollution")

        # plt.imshow(image.astype("uint8")) # Optional: display the image
        # plt.show()

        # Append the image filename and predicted class to the submit list
        file = file.split('.')[0]
        submit.append((file, PredictedClass))

# Write the predictions to a CSV file
fields = ["ImageID", 'PredictedClass']
csv_writer("TurtleVisionChallenge_Predictions_fineTuned.csv", fields, submit)

# A csv file contains the predicted classes for the test set of the TurtleVisionChallenge dataset.